In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pandas_datareader as pdr

In [4]:
#sp_500_series= web.DataReader('^GSPC', 'yahoo', start='2013-01-01', end='2019-12-31') #['Adj Close']
#sp_500_df = sp_500_series.to_frame()
#sp_500_np = sp_500_df[['Adj Close']].values

In [5]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
wiki_sp500 = table[0]
wiki_sp500.to_csv('S&P500-Info.csv')

In [6]:
sp_500_symbols = wiki_sp500['Symbol']

In [8]:
start_date = pd.Timestamp('2013-01-01')
end_date = pd.Timestamp('2019-12-31')
historical_prices = pdr.get_data_yahoo(symbols=sp_500_symbols, start=start_date, end=end_date)

/opt/anaconda3/envs/finance_gnn/lib/python3.9/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'BRK.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/opt/anaconda3/envs/finance_gnn/lib/python3.9/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'BF.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/opt/anaconda3/envs/finance_gnn/lib/python3.9/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'CARR', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/opt/anaconda3/envs/finance_gnn/lib/python3.9/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'LUMN', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/opt/anaconda3/envs/finance_gnn/lib/python3.9/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'OTIS', replacing with NaN.
  warnings.warn(msg.fo

In [11]:
prices = historical_prices['Adj Close']

Date
2013-01-02     75.612129
2013-01-03     75.524376
2013-01-04     76.082802
2013-01-07     76.178520
2013-01-08     76.186501
                 ...    
2019-12-24    168.945236
2019-12-26    168.859161
2019-12-27    169.499847
2019-12-30    168.132446
2019-12-31    168.696625
Name: MMM, Length: 1762, dtype: float64